# Data Cleaning & Exploration


### 1. The Machine Learning Process

### 2. Introduction to Matplotlib and Seaborn

### 3. Correcting Categorical Features

In [ ]:
# Import NumPy, Pandas, Pyplot (from Matplotlib), and Seaborn



In [ ]:
# Import Stock Data.csv into a new DataFrame


# Display the first row



In [ ]:
# Display the unique values in 'Sector'



### 4. Outliers & Unwanted Observations

In [ ]:
# Display the dimensions of the stock_data DataFrame



In [ ]:
# Display histograms for all continuous features



In [ ]:
# Display the box distribution of 'Beta' by 'Sector'



In [ ]:
# Display the box distribution of 'Return' by 'Sector'

